## User Story:
The IT department of _**Plain Corp**_ has implemented a custom-built _Computer Management Solution (CMS)_ which comes with a feature that reports information about computers in their environment such as metadata, installed programs and logins. The current problem faced by the IT team is that the reports created by CMS do not answer all the questions they have about the computers in their environment. Luckily, they figured out that the CMS stores its raw data in a SQLite (v3.45.0)  _database (DB)_ file and that they could write custom SQL queries to report the information not provided within the CMS reports.

## Goal:
Your job will be to update this Jupyter Notebook with the SQL queries needed to meet the IT team’s reporting needs. It is recommended to use a Database IDE to help with query writing and data exploration.

## Directions:
Download the attached zip file which contains all the files you need to complete this assignment. Below is a description of each of the three files within the attached zip file:
- **cms.sqlite**: The database you must query from to complete this assignment.
- **cms_data_dictionary.txt**: A data dictionary describing the tables within this database.
- **GA Mission - Data Analyst.ipynb**: The Jupyter Notebook file you must update to complete the assignment. Rules to abide by while completing this assignment are listed within this Jupyter Notebook.

## Our Expectations
This assignment <ins>**should take you no longer than 2 hours**</ins> to complete. Make sure your outputs are visible in a web browser when uploaded to a public repository and that they are presented in a way that is understandable by someone with some technical experience but no programming experience. Extra rules to abide by are described in the included Juptyter Notebook file.

We would prefer that you share the code with us by pushing it to a public repository in the hosting provider of your choosing. If you do not want to upload it to a public repository then you can zip up the repository folder on your machine and send it to us via email. If you go that route, please make sure that the .git folder is included.

We recommend  that you use a Database IDE (e.g., DBeaver, Datagrip, etc.) to assist with query writing and data exploration. If you have any questions, please contact us and we will do our best to clarify anything that is unclear.

## Rules:
- The code should be stored in a git repository.
- With the exception of ***Pandas***, you can use any Python Packages you like.
- Do not edit the data in the **cms.sqlite** DB file.
- Your query results must be output in a _Pandas DataFrame (PD)_.
- Questions 4 and 5 are written answer questions where no code or SQL is necessary.
- Question 6 is a written answer question that may require code or SQL depending on your answer. This question can be thought of as a chance to share any observations you've made or perform a light EDA on the data. There are no right or wrong answers for this question.

# Questions

In [219]:
import pandas as pd;
import sqlite3 as sqlite;
from IPython.display import display, HTML

 <font size='5'> 1. Count how many Admin and Regular computers are in the database.</font>

In [222]:
con = sqlite.connect("OneDrive/Documents/Galactic/cms.sqlite")
df = pd.read_sql_query("SELECT computer_type, COUNT(computer_type) AS [Count]" +
                       "FROM (SELECT DISTINCT computer_name, computer_type FROM computers)" +
                       "WHERE computer_type = 'Admin' OR computer_type = 'Regular'" +
                       "GROUP BY computer_type;", con)
blankIndex=[''] * len(df)
df.index=blankIndex
display(df)

,computer_type,Count
,Admin,8
,Regular,24


<font size='5'>2a. Identify all non-admin computers with the programs **Bitwarden**, **Quickpass** or **1password** installed, please note that the listed program names may not match exactly to the program names in the database. Be sure to include the computer names, operating system, program names and program version in the PD output.</font>

In [225]:
df = pd.read_sql_query("SELECT c.computer_name, c.computer_type, c.operating_system, ip.program_name, ip.program_version" + 
    " FROM computers c" +
    " INNER JOIN installed_programs ip ON c.computer_name = ip.computer_name" +
    " AND c.computer_type != 'Admin'" +
    " AND (ip.program_name LIKE  '%bitwarden%'" +
    	" OR ip.program_name LIKE '%quickpass%'" +
    	" OR ip.program_name LIKE '%1password%')" +
    "GROUP BY c.computer_name", con)
blankIndex=[''] * len(df)
df.index=blankIndex
print("Provided is the list of non admin computers, their operating systems, and targeted programs found on said computers along with program version.")
display(df)

Provided is the list of non admin computers, their operating systems, and targeted programs found on said computers along with program version.


,computer_name,computer_type,operating_system,program_name,program_version
,DESKTOP-O23A1M7,Service,Microsoft Windows 11 Pro,Quickpass,15.0.21225.01
,LAPTOP-55433J3D,Regular,Microsoft Windows 11 Business,Quickpass,15.0.21225.01
,PLAINCORP-1012,Regular,Microsoft Windows 11 Pro,1password,2.7.4
,PLAINCORP-1014,Regular,Microsoft Windows 11 Business,1password,2.7.4
,PLAINCORP-DRAKE,Regular,Microsoft Windows 11 Pro,Quickpass,15.0.21225.01
,PLAINCORP-WILL-P,Regular,Microsoft Windows 11 Business,Quickpass,15.0.21225.01
,Testers-Mac-mini,Regular,macOS 14.0 (23A344),/Applications/Bitwarden.app,2023.1.1
,Testers-Mini,Regular,macOS 13.3.1 (a) (22E772610a),/Applications/Bitwarden.app,2023.1.1
,WINDEV2303EVAL,Service,Microsoft Windows 11 Enterprise Evaluation,Quickpass,15.0.21225.01
,ZPPLAINCORP-MAC,Regular,macOS 13.2 (22D49),/Applications/Bitwarden.app,2023.1.1


<font size='5'>2b. Report the latest version of each program and how many computers have said programs installed.</font>

In [228]:
df = pd.read_sql_query("SELECT ip.program_name, ip.program_version, COUNT(ip.computer_name) AS computer_count" +
" FROM installed_programs ip" +
" INNER JOIN (SELECT program_name, MAX(program_version) AS max_version" +
	" FROM installed_programs" +
	" GROUP BY program_name) ip2 ON ip.program_name = ip2.program_name AND ip.program_version = ip2.max_version" +
    " AND (ip.program_name LIKE  '%bitwarden%'" +
    	" OR ip.program_name LIKE '%quickpass%'" +
    	" OR ip.program_name LIKE '%1password%')" +
	" GROUP BY ip.program_name" , con)
print("Since this question builds off of part A, I have elected to only show the results for the targeted programs in part A")
display(df)

Since this question builds off of part A, I have elected to only show the results for the targeted programs in part A


,program_name,program_version,computer_count
0,/Applications/Bitwarden.app,2023.1.1,3
1,1password,2.7.4,2
2,Quickpass,15.0.21225.01,22


<font size='5'> 2c. Report each program and the ratio of which computers out of all computers have said programs installed. Express the ratio in the format of “**X** out of **Y**“ computers and as a percentage rounded to the 2nd. Hint: To calculate the percentage, you will have to cast one of the numbers to a decimal by multiplying them by **1.0**.</font>

In [231]:
df2 = pd.read_sql_query("select COUNT(computer_name) AS computer_count FROM computers" , con)
pd.to_numeric(df.computer_count)
pd.to_numeric(df2.computer_count)
df.drop('program_version', axis=1, inplace=True)
blankIndex=[''] * len(df)
df.index=blankIndex
static_value = df2.iloc[0,0]
print(static_value)
df['total_count'] = static_value
df['ratio'] = df.computer_count.astype(str) + " out of " + df.total_count.astype(str) + " computers"
df['percentage %'] = df.computer_count.mul(1.0).div(df.total_count).mul(100).round(2)
display(df)



38


,program_name,computer_count,total_count,ratio,percentage %
,/Applications/Bitwarden.app,3,38,3 out of 38 computers,7.89
,1password,2,38,2 out of 38 computers,5.26
,Quickpass,22,38,22 out of 38 computers,57.89


<font size='5'>3a. Report which admin computers have logins on or later than May 2023, have **Python** installed on their computers and whose IP addresses are not in the IP whitelist table. Be sure to include the computer name, IP address and login timestamp; order the results by most recent date and computer name.</font>

In [234]:
df3 = pd.read_sql_query("SELECT lr.computer_name, lr.login_ip, lr.login_timestamp, c.computer_type, ip.program_name" +
" FROM login_records lr" +
" INNER JOIN (SELECT computer_name, computer_type" + 
		" FROM computers" + 
		" WHERE computer_type LIKE '%Admin%') c" + 
	" ON lr.computer_name = c.computer_name" + 
	" AND lr.login_timestamp >= '2023-05-01'" + 
" INNER JOIN (SELECT computer_name, program_name" + 
		" FROM installed_programs" + 
		" WHERE program_name LIKE '%python%') ip" + 
	" ON c.computer_name = ip.computer_name" + 
" LEFT JOIN ip_whitelist wl" + 
	" ON lr.login_ip = wl.login_ip" + 
" WHERE wl.login_ip IS NULL" + 
" GROUP BY c.computer_name" +
" ORDER BY lr.login_timestamp DESC", con)
display(df3)

,computer_name,login_ip,login_timestamp,computer_type,program_name
0,PLAINCORP-13,73.18.119.169,2023-07-19 15:56:45,Admin,Python 3.11.4 Core Interpreter (64-bit)
1,DESKTOP-AFKOOLQ,75.19.119.2,2023-06-20 15:00:10,Admin,Python 2.7.13 (64-bit)


<font size='5'>3b. Below are the requirements for the modifications you should make to the PD output from **3a**, you cannot change the SQL query to meet these requirements:</font>
- <font size='5'>Replace the 2nd and 4th octets of the IP address with **\*** (e.g., 10.\*.0.\*). Your solution must use regex.</font>
- <font size='5'>Format the login timestamp as the date only (e.g., 01/20/2024)</font>


In [299]:
formatted_df = df3.copy(deep=True)
formatted_df['login_timestamp'] = pd.to_datetime(formatted_df['login_timestamp']).dt.strftime('%m/%d/%y')
display(formatted_df)
pattern = r'(\d+)\.(\d+)\.(\d+)\.(\d+)'
formatted_df['login_ip'] = formatted_df['login_ip'].replace(pattern, r'\1.*.\3.*', regex=True)
display(formatted_df)


,computer_name,login_ip,login_timestamp,computer_type,program_name
0,PLAINCORP-13,73.18.119.169,07/19/23,Admin,Python 3.11.4 Core Interpreter (64-bit)
1,DESKTOP-AFKOOLQ,75.19.119.2,06/20/23,Admin,Python 2.7.13 (64-bit)


,computer_name,login_ip,login_timestamp,computer_type,program_name
0,PLAINCORP-13,73.*.119.*,07/19/23,Admin,Python 3.11.4 Core Interpreter (64-bit)
1,DESKTOP-AFKOOLQ,75.*.119.*,06/20/23,Admin,Python 2.7.13 (64-bit)


<font size='5'>4. Bob from the IT department noticed a SQL query was starting to get slow and since he knew a little bit of SQL from college, he decided to fix the query himself. The first thing Bob did was to reduce the number of unnecessary joins in the query, but the new query was still slow. Was Bob’s initial approach the correct one? If so, explain Bob's next steps? If not, explain what Bob should've done instead?</font>

Bob was correct in his approach. Unnecessary joins can slow down queries quite a bit. The next step would be to reduce result set size by ensuring the tables being joined are only including necessary columns, and potentially limiting the final result set through filtering.

<font size='5'>5. To improve the CMS’s backend and report capabilities, the IT department wants to provide feedback to the contractor who built the CMS solution and wants your input.  How would you improve or change the data model? If you decide to create a data diagram you must save it as a PDF file in the same folder as this Jupyter Notebook.</font>

<font size='5'>6. Did you notice anything interesting or anything wrong with the data? If so, please describe what you found and if possible, provide any related evidence.</font>